# imports

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

import pickle
import shutil
import time 
import glob
import re
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import sys
sys.path.append('./utils/')

import tensorflow as tf
print(tf.test.gpu_device_name())

import utils.utilfunc as uf
import utils.datagen as dg
from utils.models import PlanNet, RouteNet
# from utils.evaluate import eval_all_models_under


def plot_val_mae_and_loss(val_mae, val_loss, ax=None):
    if ax is None:
        fig, ax = plt.subplots(1,1)
    ax2 = ax.twinx()
    c0, c1 = 'tab:blue', 'tab:red'
    ax.plot(
        val_mae, ls='-', lw=1, color=c0, marker='.', label=None
    )
    ax2.plot(
        val_loss, ls='-', lw=1, color=c1, marker='.', label=None
    )

    ax.set_ylabel('val_mae', color=c0) 
    ax.tick_params(axis='y', labelcolor=c0)
    ax2.set_ylabel('val_loss', color=c1) 
    ax2.tick_params(axis='y', labelcolor=c1)
    ax.set_xlabel('epoch')
    if ax is None:
        plt.show()


2023-12-13 04:46:45.108015: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 04:46:45.170879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 04:46:48.006858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-12-13 04:46:51.425625: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-13 04:46:51.425701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: lido
2023-12-13 04:46:51.425711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: lido
2023-12-13 04:46:51.425866: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 470.199.2
2023-12-13 04:46:51.425899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.199.2
2023-12-13 04:46:51.425907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 470.199.2


## RouteNet

In [23]:

#PlanNetEmbGCNBatch PlanNetEmbECCBatch
tr_model = "RouteNetEmb" 
tr_conf = "./configs_univ/wifigridpert-delay-jitter-throughput-drops-labnorm-arch2.ini" #wifigrid wifigridpert nsf
"""
nsf-delay-jitter-throughput-drops.ini

nsf-delay-jitter-throughput.ini
nsf-delay-jitter-drops.ini
nsf-delay-throughput-drops.ini
nsf-jitter-throughput-drops.ini

nsf-delay.ini
nsf-jitter.ini
nsf-throughput.ini
nsf-drops.ini
"""

tr_conf_dict = uf.PathConfigParser(tr_conf).as_dict()
tr_label = "+".join(tr_conf_dict['Paths']['trainon'])
tr_dataset = os.path.basename(tr_conf_dict['Paths']['data'][0])
tr_logpath = tr_conf_dict['Paths']['logs'][0]
tr_learnrate = tr_conf_dict['LearningParams']['learning_rate']
tr_suffix = 'FromScratch'

cvfs = 4
mainfile = 'train_univ_emb_batchcpu.py'  #train_univ_emb_batchcpu train_univ_emb

train_commands = [
    f"CUDA_VISIBLE_DEVICES=-1 python {mainfile} --resume -ep 100 -es 100 --suffix {tr_suffix} -c {tr_conf} -m {tr_model} -f {f};" \
    for f in range(1,cvfs+1)
]

evalu_commands = [
    f"python utils/evaluate.py --cdir {tr_logpath}/cv{f}/{tr_model}/TRG+{tr_label}_DS+{tr_dataset}_LR{tr_learnrate:.2e}_{tr_suffix};"\
    for f in range(1,cvfs+1)
]

print('\\\n'.join(['\n'.join(
    [
        tc, 
        #ec
    ]) for tc, ec in zip(train_commands, evalu_commands)]))
# print("\\\n". join(evalu_commands))


CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --resume -ep 100 -es 100 --suffix FromScratch -c ./configs_univ/wifigridpert-delay-jitter-throughput-drops-labnorm-arch2.ini -m RouteNetEmb -f 1;\
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --resume -ep 100 -es 100 --suffix FromScratch -c ./configs_univ/wifigridpert-delay-jitter-throughput-drops-labnorm-arch2.ini -m RouteNetEmb -f 2;\
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --resume -ep 100 -es 100 --suffix FromScratch -c ./configs_univ/wifigridpert-delay-jitter-throughput-drops-labnorm-arch2.ini -m RouteNetEmb -f 3;\
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --resume -ep 100 -es 100 --suffix FromScratch -c ./configs_univ/wifigridpert-delay-jitter-throughput-drops-labnorm-arch2.ini -m RouteNetEmb -f 4;


## PLAN-Net

In [33]:

#PlanNetEmbGCNBatch PlanNetEmbECCBatch
tr_model = "PlanNetEmbGCNBatch" 
tr_conf = "./configs_univ/wifigridflow-delay-jitter-throughput-drops-labnorm-arch2.ini" #wifigrid wifigridpert nsf
"""
nsf-delay-jitter-throughput-drops.ini

nsf-delay-jitter-throughput.ini
nsf-delay-jitter-drops.ini
nsf-delay-throughput-drops.ini
nsf-jitter-throughput-drops.ini

nsf-delay.ini
nsf-jitter.ini
nsf-throughput.ini
nsf-drops.ini
"""

tr_conf_dict = uf.PathConfigParser(tr_conf).as_dict()
tr_label = "+".join(tr_conf_dict['Paths']['trainon'])
tr_dataset = os.path.basename(tr_conf_dict['Paths']['data'][0])
tr_logpath = tr_conf_dict['Paths']['logs'][0]
tr_learnrate = tr_conf_dict['LearningParams']['learning_rate']
tr_suffix = 'FromScratch_3ecc_LN'
tr_lossfunc = 'mse'#'logcosh' #

cvfs = 4
mainfile = 'train_univ_emb_batchcpu.py'  #train_univ_emb_batchcpu train_univ_emb

train_commands = [
    f"CUDA_VISIBLE_DEVICES=-1 python {mainfile} --loss {tr_lossfunc} --resume -ep 100 -es 100 --suffix {tr_suffix} -c {tr_conf} -m {tr_model} -f {f};" \
    for f in range(1,cvfs+1)
]

evalu_commands = [
    f"python utils/evaluate.py --cdir {tr_logpath}/cv{f}/{tr_model}/TRG+{tr_label}_DS+{tr_dataset}_LR{tr_learnrate:.2e}_{tr_suffix};"\
    for f in range(1,cvfs+1)
]

print('\n'.join(['\n'.join(
    [
        tc, 
        #ec
    ]) for tc, ec in zip(train_commands, evalu_commands)]))
# print("\\\n". join(evalu_commands))


CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --loss mse --resume -ep 100 -es 100 --suffix FromScratch_3ecc_LN -c ./configs_univ/wifigridflow-delay-jitter-throughput-drops-labnorm-arch2.ini -m PlanNetEmbGCNBatch -f 1;
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --loss mse --resume -ep 100 -es 100 --suffix FromScratch_3ecc_LN -c ./configs_univ/wifigridflow-delay-jitter-throughput-drops-labnorm-arch2.ini -m PlanNetEmbGCNBatch -f 2;
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --loss mse --resume -ep 100 -es 100 --suffix FromScratch_3ecc_LN -c ./configs_univ/wifigridflow-delay-jitter-throughput-drops-labnorm-arch2.ini -m PlanNetEmbGCNBatch -f 3;
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --loss mse --resume -ep 100 -es 100 --suffix FromScratch_3ecc_LN -c ./configs_univ/wifigridflow-delay-jitter-throughput-drops-labnorm-arch2.ini -m PlanNetEmbGCNBatch -f 4;


In [ ]:
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py --loss mse --resume -ep 100 -es 100 --suffix FromScratch -c ./output_univ/cv1/PlanNetEmbGCNBatch/TRG+delay+jitter+throughput+drops_DS+WifiGridR1S2St180Dr50Le2F4_LR5.00e-04_BS10_LF+mse_FromScratch/wifigrid-delay-jitter-throughput-drops-labnorm-arch2.ini  -m PlanNetEmbGCNBatch -f 1

In [176]:
tr_conf_dict['Paths']['trainon']

['jitter', 'throughput', 'drops']

## retrain

In [30]:
import warnings

#TRG+jitter+throughput+drops_DS+WifiGridR1S2St180Dr50Le2F4_LR5.00e-04_BN10_FromScratch_3ecc_LN
tr_model = "PlanNetEmbGCNBatch"
tr_conf = "./configs_univ/wifigrid-jitter-throughput-drops-labnorm-arch2.ini" # load weight from
retr_conf = "./configs_univ/wifigrid-delay-labnorm-arch2.ini"  # save results to

# original
tr_conf_dict = uf.PathConfigParser(tr_conf).as_dict()
tr_label = "+".join(tr_conf_dict['Paths']['trainon'])
tr_dataset = os.path.basename(tr_conf_dict['Paths']['data'][0])
tr_logpath = tr_conf_dict['Paths']['logs'][0]
tr_learnrate = tr_conf_dict['LearningParams']['learning_rate']
tr_bsize = '_BN'+str(tr_conf_dict['LearningParams']['batch_size']) \
    if 'batching' in tr_conf_dict['LearningParams'] and tr_conf_dict['LearningParams']['batching']\
    else ''
tr_suffix = 'FromScratch_3ecc_LN'
fn_weights = f"{tr_model}/TRG+{tr_label}_DS+{tr_dataset}_LR{tr_learnrate:.2e}{tr_bsize}_{tr_suffix}"

# retrain
retr_conf_dict = uf.PathConfigParser(retr_conf).as_dict()
retr_label = "+".join(retr_conf_dict['Paths']['trainon'])
retr_dataset = os.path.basename(retr_conf_dict['Paths']['data'][0])
retr_logpath = retr_conf_dict['Paths']['logs'][0]
retr_learnrate = retr_conf_dict['LearningParams']['learning_rate']
retr_bsize = '_BN'+str(retr_conf_dict['LearningParams']['batch_size']) \
    if 'batching' in retr_conf_dict['LearningParams'] and retr_conf_dict['LearningParams']['batching']\
    else ''
retr_suffix = 'RetrainFrom'+''.join([s.title() for s in tr_label.split('+')])+'_3ecc_LN'
fn_output = f"{tr_model}/TRG+{retr_label}_DS+{retr_dataset}_LR{retr_learnrate:.2e}{tr_bsize}_{retr_suffix}"

##
c_prefix = "CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py "
train_commands = []
for f in range(1,5):
    checkpoint_dir = f"{tr_logpath}/cv{f}/{fn_weights}"
    
    new_vers = False #legacy
    idx = (0, 1)
    if not os.path.exists(
        pkl_best := os.path.join(checkpoint_dir ,'mae_best_vals.pkl')
    ):
        if not os.path.exists(
            pkl_best := os.path.join(checkpoint_dir ,'mae_best_vals_multi.pkl')
        ):
            pkl_best = os.path.join(checkpoint_dir ,'training.log')
            new_vers = True
        idx = ('val_mae','val_loss')
    else:
        idx = (0, 1)

    if not new_vers:
        with open(pkl_best, 'rb') as h:
            models_perf_reload = pickle.load(h)    

        model_best = min(
            models_perf_reload, 
            key=lambda key: models_perf_reload[key][idx[0]] \
                if models_perf_reload[key] and key in models_perf_reload else float('inf')
        )
    else:
        dd_log = pd.read_csv(pkl_best)
        #fix some bugs
        elist = dd_log.epoch.tolist()
        i1 = [(len(elist) - elist[::-1].index(i) -1) for i in set(elist)] ##last appeareance of unique ep
        if len(i1) != len(elist):
            warnings.warn('repeated epoch numbers!')
        dd_log = dd_log.iloc[i1].reset_index(drop=True)
        i0 = len(dd_log) - dd_log.epoch.tolist()[::-1].index(0) -1 #last appeareance of 0
        if i0 != 0:
            warnings.warn('restarted epoch numbers!')
        dd_log = dd_log.iloc[i0:].reset_index(drop=True)
        model_best = f'cp-{dd_log[idx[0]].argmin():04d}.ckpt'
    
    fn_weights_n = fn_weights+'/'+model_best
    train_commands.append(
        c_prefix + f" --freeze --resume --loadweights {tr_logpath}/cv{f}/{fn_weights_n} --suffix {retr_suffix} -c {retr_conf} -m {tr_model} -f {f};" \
    )

evalu_commands = [
    " "#f"python utils/evaluate.py --cdir {tr_logpath}/cv{f}/{fn_output};"\
    for f in range(1,5)
]

print('\\\n'.join([''.join([tc, ec]) for tc, ec in zip(train_commands, evalu_commands)]))
# print("\\\n". join(evalu_commands))


CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py  --freeze --resume --loadweights ./output_univ//cv1/PlanNetEmbGCNBatch/TRG+jitter+throughput+drops_DS+WifiGridR1S2St180Dr50Le2F4_LR5.00e-04_BN10_FromScratch_3ecc_LN/cp-0037.ckpt --suffix RetrainFromJitterThroughputDrops_3ecc_LN -c ./configs_univ/wifigrid-delay-labnorm-arch2.ini -m PlanNetEmbGCNBatch -f 1; \
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py  --freeze --resume --loadweights ./output_univ//cv2/PlanNetEmbGCNBatch/TRG+jitter+throughput+drops_DS+WifiGridR1S2St180Dr50Le2F4_LR5.00e-04_BN10_FromScratch_3ecc_LN/cp-0054.ckpt --suffix RetrainFromJitterThroughputDrops_3ecc_LN -c ./configs_univ/wifigrid-delay-labnorm-arch2.ini -m PlanNetEmbGCNBatch -f 2; \
CUDA_VISIBLE_DEVICES=-1 python train_univ_emb_batchcpu.py  --freeze --resume --loadweights ./output_univ//cv3/PlanNetEmbGCNBatch/TRG+jitter+throughput+drops_DS+WifiGridR1S2St180Dr50Le2F4_LR5.00e-04_BN10_FromScratch_3ecc_LN/cp-0080.ckpt --suffix RetrainFromJitterThro

In [147]:
models_perf_reload

{'cp-0001.ckpt': {'val_loss': 0.7776424170020121,
  'val_mae': 0.9720503194950956,
  'val_reg_loss': 0.005399857968151929,
  'val_loss/jitter': 0.4321712440168354,
  'val_mae/jitter': 0.40763885682257606,
  'val_label/mean/jitter': 1.7450509944432218,
  'val_prediction/mean/jitter': 1.6000291298572686,
  'val_loss/throughput': 0.07799784854144398,
  'val_mae/throughput': 0.20735443238041532,
  'val_label/mean/throughput': 0.8715876803791499,
  'val_prediction/mean/throughput': 0.8512011399451396,
  'val_loss/drops': 0.2674730634785274,
  'val_mae/drops': 0.35705686143226545,
  'val_label/mean/drops': 0.7236476721658074,
  'val_prediction/mean/drops': 0.8019540583343813},
 'cp-best-loss.ckpt': {'val_loss': 0.7776424170020121,
  'val_mae': 0.9720503194950956,
  'val_reg_loss': 0.005399857968151929,
  'val_loss/jitter': 0.4321712440168354,
  'val_mae/jitter': 0.40763885682257606,
  'val_label/mean/jitter': 1.7450509944432218,
  'val_prediction/mean/jitter': 1.6000291298572686,
  'val_loss